In [1]:
import numpy as np
import pandas as pd

In [15]:
#Defensive Rankings from ProFootballReference
#(https://www.pro-football-reference.com/years/2021/opp.htm)
D_data ={}
overall_data = pd.DataFrame([])

for i in np.arange(1970, 2022):
    D_data[i] = pd.read_excel('Defense_Rankings.xlsx', str(i), header=[1])
    D_data[i]['Season'] = i
    overall_data = pd.concat([overall_data, D_data[i]])    
    Def = overall_data.copy()
    Def.rename(columns = {'Rk':'Def_Rk','G': 'Games', 'PA': 'Points_Allowed', 'Yds':'Tot_Yards', 
                          'Ply':'Tot_Plays', 'Y/P': 'Tot_Yards/Play', 'TO':'Takeaways', 
                          '1stD':'Tot_First_Downs', 'Cmp':'Pass_Completed', 'Att': 'Pass_Attempted',
                          'Yds.1': 'Pass_Yards', 'TD':'Pass_TD', 'NY/A':'Pass_Net/Attempt', 
                          '1stD.1':'Pass_First_Downs', 'Att.1': 'Rush_Attempted', 'Yds.2': 'Rush_Yards',
                          'Yds.2':'Rush_Yards', 'TD.1':'Rush_TD', 'Y/A': 'Rush_Yds/Attempt', 
                          '1stD.2':'Rush_First_Downs', 'Pen':'Penalities','Yds.3':'Penalty_Yards',
                          '1stPy':'First_Down_By_Penalty', 'Sc%': 'Scoring_Percentage', 
                          'TO%':'Turnover_Percentage', 'Tm':'Team'}, inplace=True)
    Def.dropna(subset=['Games'], inplace=True)
    Def = Def.reset_index(drop=True)
    
    
#Offensive Rankings from ProFootballReference 
#(https://www.pro-football-reference.com/years/2021/index.htm)    
O_data = {}
overall_data1 = pd.DataFrame([])

for i in np.arange(1970, 2022):
    O_data[i] = pd.read_excel('Offense_Rankings.xlsx', str(i), header=[1])
    O_data[i]['Season'] = i
    overall_data1 = pd.concat([overall_data1, O_data[i]])
    Off = overall_data1.copy()
    Off.rename(columns = {'Rk':'Off_Rk', 'G': 'Games', 'PF': 'Points_For', 'Yds':'Tot_Yards', 
                          'Ply':'Tot_Plays', 'Y/P': 'Tot_Yards/Play', 'TO':'Turnover', 
                          '1stD':'Tot_First_Downs', 'Cmp':'Pass_Completed', 'Att': 'Pass_Attempted',
                          'Yds.1': 'Pass_Yards', 'TD':'Pass_TD', 'NY/A':'Pass_Net/Attempt',
                          '1stD.1':'Pass_First_Downs', 'Att.1': 'Rush_Attempted', 'Yds.2': 'Rush_Yards',
                          'Yds.2':'Rush_Yards', 'TD.1':'Rush_TD', 'Y/A': 'Rush_Yds/Attempt',
                          '1stD.2':'Rush_First_Downs', 'Pen':'Penalities','Yds.3':'Penalty_Yards',
                          '1stPy':'First_Down_By_Penalty', 'Sc%': 'Scoring_Percentage', 
                          'TO%':'Turnover_Percentage', 'Tm':'Team'}, inplace=True)
    Off.dropna(subset=['Games'], inplace=True)
    Off = Off.reset_index(drop=True)

In [16]:
#Merging Offense and Defense Dataframes together
Off = Off.drop(['FL','EXP'], axis=1)
Def = Def.drop(['FL','EXP'], axis=1)

Team = pd.merge(Def, Off, how='inner', on=['Season','Team','Games'], suffixes= ('_def', '_off'))

In [17]:
#Super Bowl teams/qbs from Wikipedia (https://en.wikipedia.org/wiki/List_of_Super_Bowl_starting_quarterbacks)
SB = pd.read_csv('SuperBowlQBs.csv')

#Removing pre-1970 SB (before 1970 NFL merger)
SB.drop(SB.loc[0:3].index, inplace=True)
SB.drop(columns=['Super Bowl'], inplace=True)
SB.rename(columns={'Team.1':'Losing_Team', 'Winning QB': 'SBWin_QB', 'Team':'Win_Team',
                   'Losing QB': 'SBLosing_QB'}, inplace=True)

#Removing miscellaneous characters
SB['Win_Team'] = SB['Win_Team'].map(lambda x: str(x)[:-1])
SB['Losing_Team'] = SB['Losing_Team'].map(lambda x: str(x)[:-1])
SB['SBWin_QB'] = SB['SBWin_QB'].str.strip('*MVP‡') 
SB['SBLosing_QB'] = SB['SBLosing_QB'].str.strip('*‡')

# Changing QB isspellings
SB.loc[SB.SBWin_QB == 'eyton Manning', 'SBWin_QB'] = 'Peyton Manning'
SB.loc[SB.SBWin_QB == 'atrick Mahomes', 'SBWin_QB'] = 'Patrick Mahomes'
SB.loc[SB.SBWin_QB == 'atthew Stafford', 'SBWin_QB'] = 'Matthew Stafford'
SB.loc[SB.SBWin_QB == 'hil Simms', 'SBWin_QB'] = 'Phil Simms'
SB.loc[SB.SBWin_QB == 'ark Rypien', 'SBWin_QB'] = 'Mark Rypien'

#Preparing to merge SBWin_QB & SB_LoseQB with Team dataframeJupyter notebooks or web applications using Dash or saved as individual
SBWinners = SB[['Season', 'SBWin_QB', 'Win_Team']]
SBWinners = SBWinners.rename(columns= {'Win_Team' : 'Team'})

SBLosers = SB[['Season', 'SBLosing_QB', 'Losing_Team']]
SBLosers = SBLosers.rename(columns= {'Losing_Team' : 'Team'})

#Merging both dfs to Team dataframe
SBWinners['Season'] = SBWinners['Season'].astype(int)
SBLosers['Season'] = SBLosers['Season'].astype(int)
Team = pd.merge(pd.merge(Team, SBWinners, how='left', on=['Season','Team']), SBLosers, how='left', 
                on=['Season','Team'])

In [18]:
Team.head(5)

,Def_Rk,Team,Games,Points_Allowed,Tot_Yards_def,Tot_Plays_def,Tot_Yards/Play_def,Takeaways,Tot_First_Downs_def,Pass_Completed_def,...,Rush_TD_off,Rush_Yds/Attempt_off,Rush_First_Downs_off,Penalities_off,Penalty_Yards_off,First_Down_By_Penalty_off,Scoring_Percentage_off,Turnover_Percentage_off,SBWin_QB,SBLosing_QB
0,1.0,Minnesota Vikings,14.0,143.0,2803.0,814.0,3.4,44.0,168.0,195.0,...,16.0,3.2,98.0,60.0,631.0,16.0,NaN,NaN,NaN,NaN
1,2.0,Detroit Lions,14.0,202.0,3448.0,756.0,4.6,44.0,186.0,194.0,...,16.0,4.1,113.0,58.0,659.0,23.0,NaN,NaN,NaN,NaN
2,3.0,Los Angeles Rams,14.0,202.0,3548.0,826.0,4.3,35.0,195.0,196.0,...,12.0,4.1,93.0,88.0,959.0,11.0,NaN,NaN,NaN,NaN
3,4.0,Dallas Cowboys,14.0,221.0,3569.0,855.0,4.2,39.0,205.0,193.0,...,16.0,4.4,119.0,87.0,934.0,15.0,NaN,NaN,NaN,Craig Morton
4,5.0,St. Louis Cardinals,14.0,228.0,3869.0,894.0,4.3,35.0,242.0,183.0,...,18.0,4.7,110.0,84.0,896.0,12.0,NaN,NaN,NaN,NaN


In [38]:
#Comparing off & def rankings of superbowl teams
sbwinlose = Team[(Team['SBWin_QB'].notnull()) | (Team['SBLosing_QB'].notnull())][['Team', 'Season','Def_Rk', 'Off_Rk', 'SBWin_QB', 'SBLosing_QB']]

#average defensive & offensive rankings for sb teams 
avg_off_rank = sbwinlose['Off_Rk'].mean()
avg_def_rank = sbwinlose['Def_Rk'].mean()

#average defensive & offensive rankings for sb winners
sbwin_off_rank = sbwinlose.loc[sbwinlose['SBWin_QB'].notnull()]['Off_Rk'].mean()
sbwin_def_rank = sbwinlose.loc[sbwinlose['SBWin_QB'].notnull()]['Def_Rk'].mean()

#average defensive & offensive rankings for sb losers 
sblosers_off_rank = sbwinlose.loc[sbwinlose['SBLosing_QB'].notnull()]['Off_Rk'].mean()
sblosers_def_rank = sbwinlose.loc[sbwinlose['SBLosing_QB'].notnull()]['Def_Rk'].mean()


In [89]:
sbwinlose.tail(40)

,Team,Season,Def_Rk,Off_Rk,SBWin_QB,SBLosing_QB
901,Tampa Bay Buccaneers,2002,1.0,18.0,Brad Johnson,NaN
906,Oakland Raiders,2002,6.0,2.0,NaN,Rich Gannon
933,New England Patriots,2003,1.0,12.0,Tom Brady,NaN
942,Carolina Panthers,2003,10.0,15.0,NaN,Jake Delhomme
966,Philadelphia Eagles,2004,2.0,8.0,NaN,Donovan McNabb
967,New England Patriots,2004,3.0,4.0,Tom Brady,NaN
999,Pittsburgh Steelers,2005,3.0,9.0,Ben Roethlisberger,NaN
1003,Seattle Seahawks,2005,7.0,1.0,NaN,Matt Hasselbeck
1031,Chicago Bears,2006,3.0,2.0,NaN,Rex Grossman
1051,Indianapolis Colts,2006,23.0,3.0,Peyton Manning,NaN


In [107]:
a = sbwinlose[(sbwinlose['SBWin_QB'] == 'Tom Brady') | (sbwinlose['SBLosing_QB'] == 'Tom Brady')]
tbdef = a['Def_Rk'].mean()
tboff = a['Off_Rk'].mean()
print(f"Tom Brady's average team rankings Super Bowl defensive rank is {tbdef}:\n"
      f"Tom Brady's average team rankings Super Bowl defensive rank is {tboff}")

Tom Brady's average team rankings Super Bowl defensive rank is 5.8:
Tom Brady's average team rankings Super Bowl defensive rank is 4.2


In [50]:
b = sbwinlose[(sbwinlose['SBWin_QB'] == 'Eli Manning') | (sbwinlose['SBLosing_QB'] == 'Eli Manning')]
emdef = b['Def_Rk'].mean()
emoff = b['Off_Rk'].mean()
emoff

11.5

In [51]:
c = sbwinlose[(sbwinlose['SBWin_QB'] == 'Aaron Rodgers') | (sbwinlose['SBLosing_QB'] == 'Aaron Rodgers')]
c

,Team,Season,Def_Rk,Off_Rk,SBWin_QB,SBLosing_QB
1158,Green Bay Packers,2010,2.0,10.0,Aaron Rodgers,NaN


In [53]:
d = sbwinlose[(sbwinlose['SBWin_QB'] == 'Ben Roethlisberger') | (sbwinlose['SBLosing_QB'] == 'Ben Roethlisberger')]
brdef = d['Def_Rk'].mean()
broff = d['Off_Rk'].mean()

In [83]:
e = sbwinlose[(sbwinlose['SBWin_QB'] == 'Peyton Manning') | (sbwinlose['SBLosing_QB'] == 'Peyton Manning')]
pmdef = e['Def_Rk'].mean()
pmoff = e['Off_Rk'].mean()

In [87]:
e

,Team,Season,Def_Rk,Off_Rk,SBWin_QB,SBLosing_QB
1051,Indianapolis Colts,2006,23.0,3.0,Peyton Manning,NaN
1132,Indianapolis Colts,2009,8.0,7.0,NaN,Peyton Manning
1274,Denver Broncos,2013,22.0,1.0,NaN,Peyton Manning
1320,Denver Broncos,2015,4.0,19.0,Peyton Manning,NaN


In [70]:
gb = Team[(Team['Team'] == 'Green Bay Packers') & (Team['Season'] >= 2008)]
gb_def = gb['Def_Rk'].mean()
gb_off = gb['Off_Rk'].mean()

In [71]:
gb_off

8.071428571428571

In [72]:
gb_def

15.5

In [78]:
ne = Team[(Team['Team'] == 'New England Patriots') & (Team['Season'] >= 2001)]
ne_def = ne['Def_Rk'].mean()
ne_off = ne['Off_Rk'].mean()

In [79]:
ne_def

7.0

In [80]:
ne_off

6.095238095238095

In [63]:
Team['Team'].value_counts()

Minnesota Vikings           52
Denver Broncos              52
New Orleans Saints          52
Buffalo Bills               52
Detroit Lions               52
Green Bay Packers           52
New York Jets               52
Pittsburgh Steelers         52
New York Giants             52
San Francisco 49ers         52
Philadelphia Eagles         52
Chicago Bears               52
Dallas Cowboys              52
Atlanta Falcons             52
Cincinnati Bengals          52
Kansas City Chiefs          52
Miami Dolphins              52
New England Patriots        51
Washington Redskins         50
Cleveland Browns            49
San Diego Chargers          47
Seattle Seahawks            46
Tampa Bay Buccaneers        46
Indianapolis Colts          38
Oakland Raiders             37
Los Angeles Rams            31
Arizona Cardinals           28
Jacksonville Jaguars        27
Houston Oilers              27
Carolina Panthers           27
Baltimore Ravens            26
Tennessee Titans            23
St. Loui

In [59]:
Team.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1541 entries, 0 to 1540
Data columns (total 53 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Def_Rk                     1541 non-null   float64
 1   Team                       1541 non-null   object 
 2   Games                      1541 non-null   float64
 3   Points_Allowed             1541 non-null   float64
 4   Tot_Yards_def              1541 non-null   float64
 5   Tot_Plays_def              1541 non-null   float64
 6   Tot_Yards/Play_def         1541 non-null   float64
 7   Takeaways                  1541 non-null   float64
 8   Tot_First_Downs_def        1541 non-null   float64
 9   Pass_Completed_def         1541 non-null   float64
 10  Pass_Attempted_def         1541 non-null   float64
 11  Pass_Yards_def             1541 non-null   float64
 12  Pass_TD_def                1541 non-null   float64
 13  Int_def                    1541 non-null   float

In [37]:
sblosers_def_rank

8.384615384615385

In [8]:
#Creating QB dataframe with win/loss count
a = pd.DataFrame(Team['SBWin_QB'].value_counts())
a = a.reset_index().rename(columns= {'index':'QB'})

b = pd.DataFrame(Team['SBLosing_QB'].value_counts())
b = b.reset_index().rename(columns= {'index':'QB'})

#Merging win and losses
c = pd.merge(a, b, on='QB', how='outer')
c = c.fillna(0)

#Creating total SB and calculating winning percent
c['Total_SB'] = c['SBWin_QB'] + c['SBLosing_QB']
c['SBWin%'] = round((c['SBWin_QB']/c['Total_SB'])*100,0)
SBQB = c.copy()

In [9]:
#(https://www.pro-football-reference.com/about/comebacks.htm)
#Gamewinning drive is defined as:
#  team must win game
#  team must, at some point, have possession of the ball tied or down by one score (1-8 points) in 4th quarter or OT
#  the offensive scoring drive must conclude in the 4th quarter or OT. It can begin in the 3rd quarter so long as scrimmage plays that are part of the drive are run in the 4th quarter. This means you can't kick a FG on 1st play of 4th quarter to take lead and have it count as GWD.
#  the scoring play to put the winning points on the board must be the result of an offensive drive

#Total Game-Winning Drive by QB from Pro Football Regerence (https://www.pro-football-reference.com/leaders/gwd_career.htm)
GWD = pd.read_csv('TotalQB_GWD.csv', usecols = ['Player', 'GWD'])
GWD['Player'] = GWD['Player'].astype(str)
GWD['Player'] = GWD['Player'].str.strip('+')
GWD.rename(columns= {'Player':'QB'}, inplace=True)

#Playoff Game-Winning Drive QB from Pro Football Reference (https://www.pro-football-reference.com/leaders/gwd_career_playoffs.htm)
PGWD = pd.read_csv('PlayoffQB_GWD.csv', usecols = ['Player', 'GWD'])
PGWD['Player'] = PGWD['Player'].str.strip('+')
PGWD.rename(columns = {'GWD' : 'PGWD', 'Player':'QB'}, inplace=True)

#Merge Playoff and Total GameWinning Drive
GWinners = pd.merge(PGWD, GWD, how='outer', on='QB')

In [86]:
PGWD['PGWD'] = pd.to_numeric(PGWD['PGWD'])

ValueError: Unable to parse string "GWD" at position 35

In [10]:
#Merge Superbowl Quaterbacks to gamewinners
QBanalysis = pd.merge(SBQB, GWinners, how='left', on='QB')

#Manually inserting GWD for Joe Burrow
#(https://www.pro-football-reference.com/players/comeback.cgi?player=BurrJo01)
QBanalysis['GWD'] = np.where(QBanalysis['QB'] == 'Joe Burrow', 3, QBanalysis['GWD'])  

#Changing data types
QBanalysis['GWD'] = QBanalysis['GWD'].astype(int)
QBanalysis['PGWD'] = QBanalysis['PGWD'].fillna(0).astype(int)

In [11]:
#Playoff record by QB's
#minimum of 4 playoff games
QBrec = pd.read_excel('QB Analysis.xlsx', header=None)

#Iterating to create tabular data (coverting one column into 12 to display appropriate format)
columns = QBrec.iloc[0:12,0].tolist()
QBs = pd.DataFrame(columns = columns)

for i in np.arange(12):
    QBs.iloc[:,i] = QBrec.iloc[i::12, 0].tolist()

QBs.reset_index(inplace=True, drop=True)

#Changing column names
QBs.columns = ['QB', 'Total_PGames', 'PWins', 'Plosses', 'PTies', 'PWin%', 'HomeW', 'HomeL', 'HomeT', 
              'AwayW', 'AwayL', 'AwayT']

#Totaling home and away playoff games
QBs['Total_HomePG'] = QBs['HomeW'] + QBs['HomeL']
QBs['Total_Away PG'] = QBs['AwayW'] + QBs['AwayL']

#Converting to a percentage 
QBs['PWin%'] = QBs['PWin%']*100

#Removing unneccessary columns/characters
QBs = QBs.drop(QBs.columns[6:12], axis=1)
QBs = QBs.drop(columns= ['PTies'])
QBs = QBs.loc[QBs['QB'] != 'Player'].reset_index(drop=True)
QBs['QB'] = QBs['QB'].str.strip('*')

In [12]:
#Merge QB analysis to QBs
QBanalysis = pd.merge(QBanalysis, QBs, how='left', on=['QB'])

In [13]:
#adding playoff stats to fill nan for post-merge (for QBs whose playoff records were not recorded)

# BOB GRIESE -https://www.statmuse.com/nfl/ask/bob-griese-playoff-record#:~:text=Bob%20Griese%20had%20a%20record,the%20playoffs%20in%20his%20career.
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Bob Griese', 7, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Bob Griese', 5, QBanalysis['Plosses'])

# ROGER STAUBAUCH -https://www.statmuse.com/nfl/ask/roger-staubach-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Roger Staubach', 13, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Roger Staubach', 7, QBanalysis['Plosses'])

# JOHNNY UNITAS -https://www.statmuse.com/nfl/ask/johnny-unitas-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Johnny Unitas', 6, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Johnny Unitas', 3, QBanalysis['Plosses'])

# JIM McMAHON -https://www.statmuse.com/nfl/ask/jim-mcmahon-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Jim McMahon', 6, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Jim McMahon', 4, QBanalysis['Plosses'])

# FRAN TARKENTON -https://www.statmuse.com/nfl/ask/fran-tarkenton-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Fran Tarkenton', 6, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Fran Tarkenton', 5, QBanalysis['Plosses'])

# CRAIG MORTON -https://www.statmuse.com/nfl/ask/craig-morton-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Craig Morton', 7, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Craig Morton', 8, QBanalysis['Plosses'])

# CAM NEWTON -https://www.statmuse.com/nfl/ask/cam-newton-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Cam Newton', 3, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Cam Newton', 4, QBanalysis['Plosses'])

# REX GROSSMAN -https://www.statmuse.com/nfl/ask/rex-grossman-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Rex Grossman', 2, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Rex Grossman', 2, QBanalysis['Plosses'])

# DAVID WOODLEY -https://www.statmuse.com/nfl/ask/david-woodley-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'David Woodley', 3, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'David Woodley', 2, QBanalysis['Plosses'])

# JARED GOFF -https://www.statmuse.com/nfl/ask/jared-goff-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Jared Goff', 3, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Jared Goff', 3, QBanalysis['Plosses'])

# CHRIS CHANDLER -https://www.statmuse.com/nfl/ask/chris-chandler-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Chris Chandler', 2, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Chris Chandler', 1, QBanalysis['Plosses'])

# DREW BLEDSOE -https://www.statmuse.com/nfl/ask/drew-bledsoe-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Drew Bledsoe', 4, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Drew Bledsoe', 3, QBanalysis['Plosses'])

# NEIL O'DONNELL -https://www.statmuse.com/nfl/ask?q=neil+o%27donnell+playoff+record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == "Neil O'Donnell", 4, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == "Neil O'Donnell", 5, QBanalysis['Plosses'])

# STAN HUMPHRIES -https://www.statmuse.com/nfl/ask/stan-humphries-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Stan Humphries', 3, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Stan Humphries', 3, QBanalysis['Plosses'])

# BOOMER ESIASON -https://www.statmuse.com/nfl/ask/oomer-esiason-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Boomer Esiason', 3, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Boomer Esiason', 2, QBanalysis['Plosses'])

# KEN ANDERSON -https://www.statmuse.com/nfl/ask/ken-anderson-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Ken Anderson', 2, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Ken Anderson', 4, QBanalysis['Plosses'])

# TONY EASON -https://www.statmuse.com/nfl/ask/tony-eason-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Tony Eason', 3, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Tony Eason', 2, QBanalysis['Plosses'])

# KERRY COLLINS -https://www.statmuse.com/nfl/ask/erry-collins-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Kerry Collins', 3, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Kerry Collins', 4, QBanalysis['Plosses'])
    
# BILLY KILMER -https://www.statmuse.com/nfl/ask/billy-kilmer-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Billy Kilmer', 2, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Billy Kilmer', 5, QBanalysis['Plosses'])    

# VINCE FERRAGAMO -https://www.statmuse.com/nfl/ask/vince-ferragamo-playoff-record
QBanalysis['PWins'] = np.where(QBanalysis['QB'] == 'Vince Ferragamo', 3, QBanalysis['PWins'])
QBanalysis['Plosses'] = np.where(QBanalysis['QB'] == 'Vince Ferragamo', 4, QBanalysis['Plosses'])



In [14]:
#Calculating totals and percentages
QBanalysis['Total_PGames'] = QBanalysis['PWins'] + QBanalysis['Plosses'] 

#Playoff win %
QBanalysis['PWin%'] = (QBanalysis['PWins']/QBanalysis['Total_PGames'])*100
QBanalysis['PWin%'] = QBanalysis['PWin%'].apply(lambda x: round(x, 1))

#Playoff game winning drive % of total playoff games
QBanalysis['PGWD%'] = (QBanalysis['PGWD']/QBanalysis['Total_PGames'])*100
QBanalysis['PGWD%'] = QBanalysis['PGWD%'].apply(lambda x: round(x, 1))


QBanalysis['PGWD%perWin'] = (QBanalysis['PGWD']/QBanalysis['PWins'])*100
QBanalysis['PGWD%perWin'] = QBanalysis['PGWD%perWin'].apply(lambda x: round(x, 1))
QBanalysis

,QB,SBWin_QB,SBLosing_QB,Total_SB,SBWin%,PGWD,GWD,Total_PGames,PWins,Plosses,PWin%,Total_HomePG,Total_Away PG,PGWD%,PGWD%perWin
0,Tom Brady,7.0,3.0,10.0,70.0,14,53,47,35,12,74.5,32,15,29.8,40.0
1,Terry Bradshaw,4.0,0.0,4.0,100.0,4,23,19,14,5,73.7,12,7,21.1,28.6
2,Joe Montana,4.0,0.0,4.0,100.0,5,28,23,16,7,69.6,14,9,21.7,31.2
3,Troy Aikman,3.0,0.0,3.0,100.0,1,20,15,11,4,73.3,9,6,6.7,9.1
4,Eli Manning,2.0,0.0,2.0,100.0,5,37,12,8,4,66.7,3,9,41.7,62.5
5,Bob Griese,2.0,1.0,3.0,67.0,1,19,12,7,5,58.3,NaN,NaN,8.3,14.3
6,Ben Roethlisberger,2.0,1.0,3.0,67.0,4,53,23,13,10,56.5,12,11,17.4,30.8
7,Jim Plunkett,2.0,0.0,2.0,100.0,1,22,10,8,2,80.0,6,4,10.0,12.5
8,John Elway,2.0,3.0,5.0,40.0,6,40,21,14,7,66.7,14,7,28.6,42.9
9,Roger Staubach,2.0,2.0,4.0,50.0,2,21,20,13,7,65.0,NaN,NaN,10.0,15.4


In [18]:
QB100 = QBanalysis[QBanalysis['SBWin%'] == 100.0]
QB100

,QB,SBWin_QB,SBLosing_QB,Total_SB,SBWin%,PGWD,GWD,Total_PGames,PWins,Plosses,PWin%,Total_HomePG,Total_Away PG,PGWD%,PGWD%perWin
1,Terry Bradshaw,4.0,0.0,4.0,100.0,4,23,19,14,5,73.7,12,7,21.1,28.6
2,Joe Montana,4.0,0.0,4.0,100.0,5,28,23,16,7,69.6,14,9,21.7,31.2
3,Troy Aikman,3.0,0.0,3.0,100.0,1,20,15,11,4,73.3,9,6,6.7,9.1
4,Eli Manning,2.0,0.0,2.0,100.0,5,37,12,8,4,66.7,3,9,41.7,62.5
7,Jim Plunkett,2.0,0.0,2.0,100.0,1,22,10,8,2,80.0,6,4,10.0,12.5
11,Johnny Unitas,1.0,0.0,1.0,100.0,2,38,9,6,3,66.7,NaN,NaN,22.2,33.3
12,Joe Flacco,1.0,0.0,1.0,100.0,2,24,15,10,5,66.7,2,13,13.3,20.0
13,Drew Brees,1.0,0.0,1.0,100.0,3,53,18,9,9,50.0,11,7,16.7,33.3
14,Aaron Rodgers,1.0,0.0,1.0,100.0,2,27,21,11,10,52.4,10,11,9.5,18.2
16,Nick Foles,1.0,0.0,1.0,100.0,2,13,6,4,2,66.7,3,3,33.3,50.0


In [19]:
QBPWGD = QBanalysis[QBanalysis['PGWD%'] >= 33.3]
QBPWGD

,QB,SBWin_QB,SBLosing_QB,Total_SB,SBWin%,PGWD,GWD,Total_PGames,PWins,Plosses,PWin%,Total_HomePG,Total_Away PG,PGWD%,PGWD%perWin
4,Eli Manning,2.0,0.0,2.0,100.0,5,37,12,8,4,66.7,3,9,41.7,62.5
16,Nick Foles,1.0,0.0,1.0,100.0,2,13,6,4,2,66.7,3,3,33.3,50.0
24,Jeff Hostetler,1.0,0.0,1.0,100.0,2,10,5,4,1,80.0,3,2,40.0,50.0
30,Matthew Stafford,1.0,0.0,1.0,100.0,3,42,7,4,3,57.1,2,5,42.9,75.0
38,Colin Kaepernick,0.0,1.0,1.0,0.0,2,7,6,4,2,66.7,2,4,33.3,50.0
44,Chris Chandler,0.0,1.0,1.0,0.0,1,15,3,2,1,66.7,NaN,NaN,33.3,50.0
55,Stan Humphries,0.0,1.0,1.0,0.0,2,10,6,3,3,50.0,NaN,NaN,33.3,66.7
58,Joe Burrow,0.0,1.0,1.0,0.0,2,3,4,3,1,75.0,2,2,50.0,66.7


In [12]:
QBanalysis.sort_values(by='SBWin_QB')

,QB,SBWin_QB,SBLosing_QB,Total_SB,SBWin%,PGWD,GWD,Total_PGames,PWins,Plosses,PWin%,Total_HomePG,Total_Away PG,PGWD%,PGWD%perWin
58,Joe Burrow,0.0,1.0,1.0,0.0,2,3,4,3,1,75.0,2,2,50.0,66.7
31,Jim Kelly,0.0,4.0,4.0,0.0,1,28,17,9,8,52.9,11,6,5.9,11.1
32,Fran Tarkenton,0.0,3.0,3.0,0.0,1,33,11,6,5,54.5,NaN,NaN,9.1,16.7
33,Craig Morton,0.0,2.0,2.0,0.0,1,21,15,7,8,46.7,NaN,NaN,6.7,14.3
34,Jake Delhomme,0.0,1.0,1.0,0.0,1,24,8,5,3,62.5,2,6,12.5,20.0
35,Donovan McNabb,0.0,1.0,1.0,0.0,1,24,16,9,7,56.2,9,7,6.2,11.1
36,Matt Hasselbeck,0.0,1.0,1.0,0.0,2,24,11,5,6,45.5,6,5,18.2,40.0
37,Rex Grossman,0.0,1.0,1.0,0.0,1,9,4,2,2,50.0,NaN,NaN,25.0,50.0
38,Colin Kaepernick,0.0,1.0,1.0,0.0,2,7,6,4,2,66.7,2,4,33.3,50.0
39,Cam Newton,0.0,1.0,1.0,0.0,0,20,7,3,4,42.9,NaN,NaN,0.0,0.0


In [13]:
#count of duplicates
len(GWinners)-len(GWinners.drop_duplicates('QB'))

13